In [22]:
import sqlite3
import main

# get the JSON data of friends' activity from the main module
friends_activity_json = main.get_friends_activity_json()

# define a function to store the user data to a database
def store_user_data_to_database(friends_activity_json, database_name='friends_activity.db'):
    # connect to the database with the given name or create a new one if it does not exist
    conn = sqlite3.connect(database_name)
    cur = conn.cursor()

    # create a table for users with columns for user_id, user_url, user_name and user_image_url
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        user_url TEXT NOT NULL,
        user_name TEXT NOT NULL,
        user_image_url TEXT NOT NULL
    )
    ''')

    # create a table for albums with columns for album_id, album_uri and album_name
    cur.execute('''CREATE TABLE IF NOT EXISTS albums(
        album_id INTEGER PRIMARY KEY,
        album_uri TEXT NOT NULL,
        album_name TEXT NOT NULL
    )
    ''')

    # create a table for artists with columns for artist_id, artist_uri and artist_name
    cur.execute('''CREATE TABLE IF NOT EXISTS artists(
        artist_id INTEGER PRIMARY KEY,
        artist_uri TEXT NOT NULL,
        artist_name TEXT NOT NULL
    )
    ''')

    # create a table for tracks with columns for track_id, track_uri, track_name, track_image_url, album_id and artist_id
    # add foreign key constraints to reference the album_id and artist_id from the albums and artists tables respectively
    cur.execute('''CREATE TABLE IF NOT EXISTS tracks (
        track_id INTEGER PRIMARY KEY,
        track_uri TEXT NOT NULL,
        track_name TEXT NOT NULL,
        track_image_url TEXT NOT NULL,
        album_id INTEGER NOT NULL,
        artist_id INTEGER NOT NULL,
        FOREIGN KEY (album_id) REFERENCES albums(album_id),
        FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
    )
    ''')

    # create a table for context with columns for context_id, context_uri, context_name and context_index
    cur.execute('''CREATE TABLE IF NOT EXISTS context(
        context_id INTEGER PRIMARY KEY,
        context_uri TEXT NOT NULL,
        context_name TEXT NOT NULL,
        context_index INTEGER NOT NULL
    )
    ''')

    # create a table for streamings with columns for user_id, track_id and timestamp
    # add foreign key constraints to reference the user_id and track_id from the users and tracks tables respectively
    cur.execute('''CREATE TABLE IF NOT EXISTS streamings(
        user_id INTEGER NOT NULL,
        track_id INTEGER NOT NULL,
        timestamp TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (track_id) REFERENCES tracks(track_id)
    )
    '''
    )

    # loop through the JSON data of friends' activity
    for data in friends_activity_json['friends']:
        # get the user data from the JSON object
        print(data)
        user_url = data['user']['uri']
        user_name = data['user']['name']
        user_image_url = data['user']['imageUrl']

        # check if the user already exists in the users table by querying the user_url column
        cur.execute("SELECT user_id FROM users WHERE user_url = ?", (user_url,))
        user_id = cur.fetchone()

        # if the query returns None, it means the user does not exist in the table
        if user_id is None:
            # insert a new row into the users table with the user data and get the generated user_id value
            cur.execute("INSERT INTO users (user_url, user_name, user_image_url) VALUES (?, ?, ?)", (user_url, user_name, user_image_url))
            conn.commit()
            user_id = cur.lastrowid
        else:
            # if the query returns a tuple, it means the user already exists in the table and extract the first element of the tuple as the user_id value
            user_id = user_id[0]

        # get the album data from the JSON object
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']

        # check if the album already exists in the albums table by querying the album_uri column
        cur.execute("SELECT album_id FROM albums WHERE album_uri = ?", (album_uri,))
        album_id = cur.fetchone()

        # if the query returns None, it means the album does not exist in the table
        if album_id is None:
            # insert a new row into the albums table with the album data and get the generated album_id value
            cur.execute("INSERT INTO albums (album_uri, album_name) VALUES (?, ?)", (album_uri, album_name))
            conn.commit()
            album_id = cur.lastrowid
        else:
            # if the query returns a tuple, it means the album already exists in the table and extract the first element of the tuple as the album_id value
            album_id = album_id[0]

        # get the artist data from the JSON object
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']

        # check if the artist already exists in the artists table by querying the artist_uri column
        cur.execute("SELECT artist_id FROM artists WHERE artist_uri = ?", (artist_uri,))
        artist_id = cur.fetchone()

        # if the query returns None, it means the artist does not exist in the table
        if artist_id is None:
            # insert a new row into the artists table with the artist data and get the generated artist_id value
            cur.execute("INSERT INTO artists (artist_uri, artist_name) VALUES (?, ?)", (artist_uri, artist_name))
            conn.commit()
            artist_id = cur.lastrowid
        else:
            # if the query returns a tuple, it means the artist already exists in the table and extract the first element of the tuple as the artist_id value
            artist_id = artist_id[0]

        # get the track data from the JSON object
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image_url = data['track']['imageUrl']

        # check if the track already exists in the tracks table by querying the track_uri column
        cur.execute("SELECT track_id FROM tracks WHERE track_uri = ?", (track_uri,))
        track_id = cur.fetchone()

        # if the query returns None, it means the track does not exist in the table
        if track_id is None:
            # insert a new row into the tracks table with the track data and get the generated track_id value
            cur.execute("INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) VALUES (?, ?, ?, ?, ?)", (track_uri, track_name, track_image_url, album_id, artist_id))
            conn.commit()
            track_id = cur.lastrowid
        else:
            # if the query returns a tuple, it means the track already exists in the table and extract the first element of the tuple as the track_id value
            track_id = track_id[0]

        # get the context data from the JSON object
        context_uri = data['track']['context']['uri']
        context_name = data['track']['context']['name']
        context_index = data['track']['context']['index']

        # check if the context already exists in the context table by querying the context_uri column
        cur.execute("SELECT context_id FROM context WHERE context_uri = ?", (context_uri,))
        context_id = cur.fetchone()

        # if the query returns None, it means the context does not exist in the table
        if context_id is None:
            # insert a new row into the context table with the context data and get the generated context_id value
            cur.execute("INSERT INTO context (context_uri, context_name, context_index) VALUES (?, ?, ?)", (context_uri, context_name, context_index))
            conn.commit()
            context_id = cur.lastrowid
        else:
            # if the query returns a tuple, it means the context already exists in the table and extract the first element of the tuple as the context_id value
            context_id = context_id[0]

        # get the timestamp data from the JSON object
        timestamp = data['timestamp']

        # check if there is already a streaming with the same timestamp in the streamings table by querying the timestamp column
        cur.execute("SELECT * FROM streamings WHERE timestamp = ?", (timestamp,))
        streaming = cur.fetchone()

        # if the query returns None, it means there is no streaming with the same timestamp in the table
        if streaming is None:
            # insert a new row into the streamings table with the user_id, track_id and timestamp values
            cur.execute("INSERT INTO streamings (user_id, track_id, timestamp) VALUES (?, ?, ?)", (user_id, track_id, timestamp))
            conn.commit()

def print_the_data_from_the_database():
    #connect to the database
    conn = sqlite3.connect('friends_activity.db')
    cur = conn.cursor()

    #select all the data from the users table
    cur.execute("SELECT * FROM users")
    users = cur.fetchall()

    #print the user data
    print("Users:")
    for user in users:
        print(user)

    #select all the data from the albums table
    cur.execute("SELECT * FROM albums")
    albums = cur.fetchall()

    #print the album data
    print("Albums:")
    for album in albums:
        print(album)

    #select all the data from the artists table
    cur.execute("SELECT * FROM artists")
    artists = cur.fetchall()

    #print the artist data
    print("Artists:")
    for artist in artists:
        print(artist)

    #select all the data from the tracks table
    cur.execute("SELECT * FROM tracks")
    tracks = cur.fetchall()

    #print the track data
    print("Tracks:")
    for track in tracks:
        print(track)

    #select all the data from the context table
    cur.execute("SELECT * FROM context")
    context = cur.fetchall()

    #print the context data
    print("Context:")
    for c in context:
        print(c)

    #select all the data from the streamings table
    cur.execute("SELECT * FROM streamings")
    streamings = cur.fetchall()

    #print the streaming data
    print("Streamings:")
    for streaming in streamings:
        print(streaming)



store_user_data_to_database(friends_activity_json)



{'timestamp': 1684069017858, 'user': {'uri': 'spotify:user:sonemic.com', 'name': 'sonemic.com', 'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'}, 'track': {'uri': 'spotify:track:6hznbfFE7idaaS7v2Solxv', 'name': 'Delial', 'imageUrl': 'http://i.scdn.co/image/ab67616d0000b2733a21b73579f7808ff6980f2a', 'album': {'uri': 'spotify:album:3AxEzgmziHeFJakjU2IpMo', 'name': 'In The Fishtank 14'}, 'artist': {'uri': 'spotify:artist:2vsXeWGC8rILp3rpSN2Fyk', 'name': 'ISIS'}, 'context': {'uri': 'spotify:album:7JjgvQWsLpIAvwgj2QtxAP', 'name': "The Bees Made Honey In The Lion's Skull", 'index': 0}}}
{'timestamp': 1684074584415, 'user': {'uri': 'spotify:user:earm28v0wpfycco58h1e64qi4', 'name': 'HuzgOnline', 'imageUrl': 'https://i.scdn.co/image/ab67757000003b8243ad9adbb38d891c9c34b214'}, 'track': {'uri': 'spotify:track:6E2hXlmgxJpVgZtNeVQ4aO', 'name': 'Head', 'imageUrl': 'http://i.scdn.co/image/ab67616d0000b2737ff32c46fd3ad7b86994764e', 'album': {'uri': 'spotify:album:6XgGYXz

In [23]:
print_the_data_from_the_database()

Users:
(1, 'spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f')
(2, 'spotify:user:earm28v0wpfycco58h1e64qi4', 'HuzgOnline', 'https://i.scdn.co/image/ab67757000003b8243ad9adbb38d891c9c34b214')
(3, 'spotify:user:13v0l2zagkat9thdoqngm8ki8', 'maja', 'https://i.scdn.co/image/ab67757000003b822630b0839d17ab28f6be5b99')
(4, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')
(5, 'spotify:user:homestar99', 'Eliott', 'https://i.scdn.co/image/ab67757000003b8233e21b088263fa0f21ac5f2b')
(6, 'spotify:user:02aj59ugjm84jmkqaimbjl2hb', 'gaurisha', 'https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab')
(7, 'spotify:user:gntab9tp1cc5qipthodlvvsm3', 'suchita', 'https://i.scdn.co/image/ab67757000003b829dc4867ae0babe8b2d882f5a')
(8, 'spotify:user:jaime33', 'Irles Music', 'https://i.scdn.co/image/ab67757000003b82cdbf4610bef6efb87adb4daf')
(9, 'spotify:user:miles_lamborgenie

In [9]:
for i in range(0, len(friends_activity_json)):
    # print all the users name
    print(friends_activity_json[0]['user']['name'])

sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com
sonemic.com


In [2]:

friends_activity_json

[{'timestamp': 1684069017858,
  'user': {'uri': 'spotify:user:sonemic.com',
   'name': 'sonemic.com',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
  'track': {'uri': 'spotify:track:6hznbfFE7idaaS7v2Solxv',
   'name': 'Delial',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b2733a21b73579f7808ff6980f2a',
   'album': {'uri': 'spotify:album:3AxEzgmziHeFJakjU2IpMo',
    'name': 'In The Fishtank 14'},
   'artist': {'uri': 'spotify:artist:2vsXeWGC8rILp3rpSN2Fyk', 'name': 'ISIS'},
   'context': {'uri': 'spotify:album:7JjgvQWsLpIAvwgj2QtxAP',
    'name': "The Bees Made Honey In The Lion's Skull",
    'index': 0}}},
 {'timestamp': 1684074584415,
  'user': {'uri': 'spotify:user:earm28v0wpfycco58h1e64qi4',
   'name': 'HuzgOnline',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b8243ad9adbb38d891c9c34b214'},
  'track': {'uri': 'spotify:track:6E2hXlmgxJpVgZtNeVQ4aO',
   'name': 'Head',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b2737ff32c46fd

In [14]:
from datetime import datetime
import time
import sqlite3
import main
import json

# milliseconds to datetime
def milliseconds_to_datetime(milliseconds):
    return datetime.fromtimestamp(milliseconds / 1000.0)

# get the friends activity json object from the main.py
friends_activity_json = main.get_friends_activity_json()

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Use a context manager to connect to the SQLite database.
with sqlite3.connect('friends_activity.sqlite') as conn:
#
    # Connect to the SQLite database.
    cur = conn.cursor()
    # create the users table with user_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY,
            user_uri TEXT NOT NULL UNIQUE,
            user_name TEXT NOT NULL,
            user_imageUrl TEXT NOT NULL
        )
    ''')

    # create the tracks table with track_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id INTEGER PRIMARY KEY,
            track_uri TEXT NOT NULL UNIQUE,
            track_name TEXT NOT NULL,
            track_imageUrl TEXT NOT NULL,
            track_album_uri TEXT NOT NULL,
            track_album_name TEXT NOT NULL,
            track_artist_uri TEXT NOT NULL,
            track_artist_name TEXT NOT NULL
        )
    ''')

    # create the contexts table with context_uri as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS contexts (
            context_uri TEXT PRIMARY KEY,
            context_name TEXT NOT NULL
        )
    ''')

    # create the listenings table with user_id and track_id as the foreign keys
    cur.execute('''
        CREATE TABLE IF NOT EXISTS listenings (
            user_id INTEGER NOT NULL,
            track_id INTEGER NOT NULL,
            context_id TEXT NOT NULL,
            timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
            track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
            FOREIGN KEY (user_id) REFERENCES users (user_id),
            FOREIGN KEY (track_id) REFERENCES tracks (track_id),
            FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
        )
    ''')

    # add friends activity to the database
    for friend in friends_activity_json['friends']:
        # print(friend[])
        # add user to the database

        # add user_uri, user_name and user_imageUrl if the user is not in the database

        cur.execute("""
            INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
            VALUES (?, ?, ?)
        """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))

        # get the user_id of the inserted user
        cur.execute("""
            SELECT user_id FROM users WHERE user_uri = ?
        """, (friend['user']['uri'],))
        user_id = cur.fetchone()[0]
        # add track to the database
        cur.execute("""
            INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
        # get the track_id of the inserted track
        cur.execute("""
            SELECT track_id FROM tracks WHERE track_uri = ?
        """, (friend['track']['uri'],))
        track_id = cur.fetchone()[0]
        # add context to the database
        cur.execute("""
            INSERT OR IGNORE INTO contexts (context_uri, context_name)
            VALUES (?, ?)
        """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        result = cur.fetchone()
        if result:
            # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
            old_timestamp = result[0]
            new_timestamp = friend['timestamp']
            cur.execute("""
                UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
            """, (new_timestamp, user_id, track_id, old_timestamp))
        else:
            # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
            context_id = friend['track']['context']['uri']
            timestamp = friend['timestamp']
            cur.execute("""
                INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
                VALUES (?, ?, ?, ?, 1)
            """, (user_id, track_id, context_id, timestamp))
        
        # previous_timestamps = cur.fetchall()
        
    users = cur.fetchall()

    for i in users:
        print(i)


In [22]:
#select all the data from the users table
cur.execute("SELECT * FROM users")
users = cur.fetchall()

#print the user data
print("Users:")
for user in users:
    print(user)

#select all the data from the albums table
cur.execute("SELECT * FROM albums")
albums = cur.fetchall()

#print the album data
print("Albums:")
for album in albums:
    print(album)

#select all the data from the artists table
cur.execute("SELECT * FROM artists")
artists = cur.fetchall()

#print the artist data
print("Artists:")
for artist in artists:
    print(artist)

#select all the data from the tracks table
cur.execute("SELECT * FROM tracks")
tracks = cur.fetchall()

#print the track data
print("Tracks:")
for track in tracks:
    print(track)

#select all the data from the context table
cur.execute("SELECT * FROM context")
context = cur.fetchall()

#print the context data
print("Context:")
for c in context:
    print(c)

#select all the data from the streamings table
cur.execute("SELECT * FROM streamings")
streamings = cur.fetchall()

#print the streaming data
print("Streamings:")
for streaming in streamings:
    print(streaming)



Users:
(1, 'spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f')
(2, 'spotify:user:earm28v0wpfycco58h1e64qi4', 'HuzgOnline', 'https://i.scdn.co/image/ab67757000003b8243ad9adbb38d891c9c34b214')
(3, 'spotify:user:13v0l2zagkat9thdoqngm8ki8', 'maja', 'https://i.scdn.co/image/ab67757000003b822630b0839d17ab28f6be5b99')
(4, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')
(5, 'spotify:user:homestar99', 'Eliott', 'https://i.scdn.co/image/ab67757000003b8233e21b088263fa0f21ac5f2b')
(6, 'spotify:user:02aj59ugjm84jmkqaimbjl2hb', 'gaurisha', 'https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab')
(7, 'spotify:user:gntab9tp1cc5qipthodlvvsm3', 'suchita', 'https://i.scdn.co/image/ab67757000003b829dc4867ae0babe8b2d882f5a')
(8, 'spotify:user:jaime33', 'Irles Music', 'https://i.scdn.co/image/ab67757000003b82cdbf4610bef6efb87adb4daf')
(9, 'spotify:user:miles_lamborgenie

The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.
The user with user_id 1 has listened to the track with track_id 1 0 times.


In [2]:
OLD CODE
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
# #
#     # Connect to the SQLite database.
#     cur = conn.cursor()
#     # create the users table with user_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_id INTEGER PRIMARY KEY,
#             user_uri TEXT NOT NULL UNIQUE,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     # create the tracks table with track_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_id INTEGER PRIMARY KEY,
#             track_uri TEXT NOT NULL UNIQUE,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     # create the contexts table with context_uri as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # create the listenings table with user_id and track_id as the foreign keys
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id INTEGER NOT NULL,
#             track_id INTEGER NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
#             FOREIGN KEY (user_id) REFERENCES users (user_id),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # print(friend[])
#         # add user to the database

#         # add user_uri, user_name and user_imageUrl if the user is not in the database
        

#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))

#         # get the user_id of the inserted user
#         cur.execute("""
#             SELECT user_id FROM users WHERE user_uri = ?
#         """, (friend['user']['uri'],))
#         user_id = cur.fetchone()[0]
#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # get the track_id of the inserted track
#         cur.execute("""
#             SELECT track_id FROM tracks WHERE track_uri = ?
#         """, (friend['track']['uri'],))
#         track_id = cur.fetchone()[0]
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
#         result = cur.fetchone()
#         if result:
#             # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
#             old_timestamp = result[0]
#             new_timestamp = friend['timestamp']
#             cur.execute("""
#                 UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
#             """, (new_timestamp, user_id, track_id, old_timestamp))
#         else:
#             # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
#             context_id = friend['track']['context']['uri']
#             timestamp = friend['timestamp']
#             cur.execute("""
#                 INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#                 VALUES (?, ?, ?, ?, 1)
#             """, (user_id, track_id, context_id, timestamp))
        
#         # previous_timestamps = cur.fetchall()
        
#     users = cur.fetchall()

#     for i in users:
#         print(i)


SyntaxError: invalid syntax (773587114.py, line 1)

In [13]:
# print the tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

# add data to the tables
for user in friends_activity_json['friends']:

conn.commit()

# print the tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
c:\Users\saket\Documents\GitHub\Pyhton\Project Music\spotify api\test.ipynb Cell 3 in ()
      5 # add data to the tables
      6 for user in friends_activity_json['friends']:
----> 7     cur.execute('''INSERT INTO users(user_id, user_url, user_name, user_image_url) VALUES (?,?,?,?)''', (user['id'], user['url'], user['name'], user['imageUrl']))
      8     for streaming in user['streamings']:
      9         cur.execute('''INSERT INTO streamings(user_id, track_id, timestamp) VALUES (?,?,?)''', (user['id'], streaming['track']['id'], streaming['timestamp']))

KeyError: 'id'


IndentationError: expected an indented block (1393796997.py, line 8)

In [4]:

    # # Connect to the SQLite database.
    # cur = conn.cursor()
    # # create the users table with user_id as the primary key
    # cur.execute('''
    #     CREATE TABLE IF NOT EXISTS users (
    #         user_id INTEGER PRIMARY KEY,
    #         user_uri TEXT NOT NULL UNIQUE,
    #         user_name TEXT NOT NULL,
    #         user_imageUrl TEXT NOT NULL
    #     )
    # ''')

    # # create the tracks table with track_id as the primary key
    # cur.execute('''
    #     CREATE TABLE IF NOT EXISTS tracks (
    #         track_id INTEGER PRIMARY KEY,
    #         track_uri TEXT NOT NULL UNIQUE,
    #         track_name TEXT NOT NULL,
    #         track_imageUrl TEXT NOT NULL,
    #         track_album_uri TEXT NOT NULL,
    #         track_album_name TEXT NOT NULL,
    #         track_artist_uri TEXT NOT NULL,
    #         track_artist_name TEXT NOT NULL
    #     )
    # ''')

    # # create the contexts table with context_uri as the primary key
    # cur.execute('''
    #     CREATE TABLE IF NOT EXISTS contexts (
    #         context_uri TEXT PRIMARY KEY,
    #         context_name TEXT NOT NULL
    #     )
    # ''')

    # # create the listenings table with user_id and track_id as the foreign keys
    # cur.execute('''
    #     CREATE TABLE IF NOT EXISTS listenings (
    #         user_id INTEGER NOT NULL,
    #         track_id INTEGER NOT NULL,
    #         context_id TEXT NOT NULL,
    #         timestamp INTEGER NOT NULL CHECK (timestamp > 0), -- store timestamps as UNIX epoch values
    #         track_listens_count INTEGER NOT NULL CHECK (track_listens_count > 0),
    #         FOREIGN KEY (user_id) REFERENCES users (user_id),
    #         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
    #         FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
    #     )
    # ''')

    # add friends activity to the database
    for friend in friends_activity_json['friends']:
        # print(friend[])
        # add user to the database
        cur.execute("""
            INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
            VALUES (?, ?, ?) 
        """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        # get the user_id of the inserted user
        cur.execute("""
            SELECT user_id FROM users WHERE user_uri = ?
        """, (friend['user']['uri'],))
        user_id = cur.fetchone()[0]
        # add track to the database
        cur.execute("""
            INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
        # get the track_id of the inserted track
        cur.execute("""
            SELECT track_id FROM tracks WHERE track_uri = ?
        """, (friend['track']['uri'],))
        track_id = cur.fetchone()[0]
        # add context to the database
        cur.execute("""
            INSERT OR IGNORE INTO contexts (context_uri, context_name)
            VALUES (?, ?)
        """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        result = cur.fetchone()
        if result:
            # if yes, update the listenings table with the new timestamp and increment the track_listens_count by 1
            old_timestamp = result[0]
            new_timestamp = friend['timestamp']
            cur.execute("""
                UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ? AND timestamp = ?
            """, (new_timestamp, user_id, track_id, old_timestamp))
        else:
            # if no, insert a new row into the listenings table with the context_id, timestamp and track_listens_count as 1
            context_id = friend['track']['context']['uri']
            timestamp = friend['timestamp']
            cur.execute("""
                INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
                VALUES (?, ?, ?, ?, 1)
            """, (user_id, track_id, context_id, timestamp))
        
        # previous_timestamps = cur.fetchall()
        
    users = cur.fetchall()

    for i in users:
        print(i)


OperationalError: table users has no column named user_uri

In [45]:

def user_id_to_user_uri(users = ["gntab9tp1cc5qipthodlvvsm3"]):
    """
    Description
    -----------
    Convert a list of user ids to user uris.

    Parameters
    ----------
    users : list
        A list of user ids.

    Returns
    -------
    user_uris : list
        A list of user uris.
    """
    user_ids = ["spotify:user:" + user_id for user_id in user_ids]
    return user_ids

In [47]:
# from main import *


# def create_file(filename):
#     # Create the files if they don't exist
#     if not os.path.exists(filename):
#         with open(filename, 'w') as f:
#             f.write('')


# def write_to_csv(filename, data):
#     # Write the data to the csv file
#     with open(filename, 'a', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(data)

# def main():
#     # Get the friends activity json
#     friends_activity_json = get_friends_activity_json()

#     # Get the current time
#     current_time = datetime.datetime.now()

#     # Iterate through the friends activity
#     for friend in friends_activity_json['friends']:
#         # Check if the user uri is the same as the user uri
#         if friend['user']['uri'] in user_id_to_user_uri():
#             # The id of the particular track the particular user is listening to
#             track_id = friend['track']['uri'].split(":")[-1]

#             # Check if the track is same as the previous track don't add it to the csv file
#             if track_id not in [i['track']['uri'].split(":")[-1] for i in friend_activity_csv['track_uri'].iloc[-10:]]:
#                 # Write the data to the csv file
#                 write_to_csv('friend_activity.csv', [friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

#                 # Get all the playlist tracks from the playlist
#                 the_playlist_tracks = get_playlist_tracks_retry(playlist_id)

#                 # If the track is not in the playlist add it to the playlist
#                 if track_id not in [i['track']['uri'].split(":")[-1] for i in the_playlist_tracks]:
#                     add_track_to_playlist(playlist_id, track_id)

#     # Wait for 30 seconds
#     time.sleep(30)


# if __name__ == '__main__':
#     main()


UnboundLocalError: local variable 'user_ids' referenced before assignment

In [19]:


# def get_current_time():
#     # Get the current time.
#     return datetime.datetime.now()


# def check_if_track_is_new(track_id, friend_activity_csv):
#     # Check if the track is the same as the previous track.
#     if track_id not in [i['track_uri'].split(":")[-1] for i in friend_activity_csv['track_uri'].iloc[-10:]]:
#         return True
#     else:
#         return False


# def write_to_csv(filename, data):
#     # Write the data to the CSV file.
#     with open(filename, 'a', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(data)


# def get_playlist_tracks(playlist_id):
#     # Get all the playlist tracks.
#     while True:
#         try:
#             return sp.user_playlist_tracks(username, playlist_id)
#         except requests.exceptions.ConnectionError:
#             print('ConnectionError, retrying...')
#             time.sleep(30)


# def add_track_to_playlist(playlist_id, track_id):
#     # Add the track to the playlist.
#     while True:
#         try:
#             sp.user_playlist_add_tracks(username, playlist_id, [track_id])
#             break
#         except requests.exceptions.ConnectionError:
#             print('ConnectionError, retrying...')
#             time.sleep(30)


# def main():
#     # Get the friends activity data.
#     friends_activity_data = get_friends_activity_json()

#     # Get the current time.
#     current_time = get_current_time()

#     # Iterate through the friends activity data.
#     for friend in friends_activity_data['friends']:
#         # Check if the user URI is the same as the user URI.
#         if friend['user']['uri'] in user_ids:
#             # Get the track ID.
#             track_id = friend['track']['uri'].split(":")[-1]

#             # Check if the track is the same as the previous track.
#             if check_if_track_is_new(track_id, friend_activity_csv):
#                 # Write the data to the CSV file.
#                 write_to_csv('friend_activity.csv', [friend['user']['uri'], friend['track']['uri'], friend['timestamp'], current_time])

#                 # Get all the playlist tracks.
#                 the_playlist_tracks = get_playlist_tracks(playlist_id)

#                 # If the track is not in the playlist, add it to the playlist.
#                 if track_id not in [i['track']['uri'].split(":")[-1] for i in the_playlist_tracks]:
#                     add_track_to_playlist(playlist_id, track_id)

#     # Wait for 30 seconds.
#     time.sleep(30)
# main()

In [18]:
# for i in friends_activity_json['friends']:
#     print(i['timestamp'])

In [81]:
# import sqlite3

# # connect to the database
# conn = sqlite3.connect('friends_activity.sqlite')

# # get a cursor to the database
# cur = conn.cursor()

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS users (
#         user_id INTEGER PRIMARY KEY,
#         user_uri TEXT NOT NULL,
#         user_name TEXT NOT NULL,
#         user_imageUrl TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS tracks (
#         track_id INTEGER PRIMARY KEY,
#         track_uri TEXT NOT NULL,
#         track_name TEXT NOT NULL,
#         track_imageUrl TEXT NOT NULL,
#         track_album_uri TEXT NOT NULL,
#         track_album_name TEXT NOT NULL,
#         track_artist_uri TEXT NOT NULL,
#         track_artist_name TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS contexts (
#         context_uri TEXT PRIMARY KEY,
#         context_name TEXT NOT NULL
#     )
# ''')

# # create a table if it doesn't already exist
# cur.execute('''
#     CREATE TABLE IF NOT EXISTS listenings (
#         user_id INTEGER NOT NULL,
#         track_id INTEGER NOT NULL,
#         context_id TEXT NOT NULL,
#         timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#         track_listens_count INTEGER NOT NULL,
#         FOREIGN KEY (user_id) REFERENCES users (user_id),
#         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#         FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#     )
# ''')
# # add friends activity to the database
# for friend in friends_activity_json['friends']:
#     # check if the user already exists in the database
#     cur.execute("""
#         SELECT user_id FROM users WHERE user_uri = ?
#     """, (friend['user']['uri'],))
#     user_id = cur.fetchone()

#     # if the user does not exist, add them to the database
#     if user_id is None:
#         cur.execute("""
#             INSERT INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         user_id = cur.lastrowid

#     # check if the track already exists in the database
#     cur.execute("""
#         SELECT track_id FROM tracks WHERE track_uri = ?
#     """, (friend['track']['uri'],))
#     track_id = cur.fetchone()

#     # if the track does not exist, add it to the database
#     if track_id is None:
#         cur.execute("""
#             INSERT INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album_name'], friend['track']['artist']['uri'], friend['track']['artist_name']))
#         track_id = cur.lastrowid

#     # add listening to the database
#     cur.execute("""
#         INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#         VALUES (?, ?, NULL, ?, 1)
#     """, (user_id, track_id, friend['timestamp']))

# # commit the changes to the database
# conn.commit()

# # close the connection to the database
# conn.close()


InterfaceError: Error binding parameter 0 - probably unsupported type.

In [7]:

# # Create a SQLite database file.
# conn = sqlite3.connect('friends_activity.sqlite')

# # Connect to the SQLite database.
# cur = conn.cursor()

# # print all the users from the database
# cur.execute("""
#     SELECT * FROM users
# """)
# users = cur.fetchall()

# print("users: \n user_uri, user_name, user_imageUrl")
# for i in users:
#     print(i)

# # print all the tracks from the database
# cur.execute("""
#     SELECT * FROM tracks
# """)
# tracks = cur.fetchall()

# print("tracks: \n track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name")
# for i in tracks:
#     print(i)

# # print all the contexts from the database
# cur.execute("""
#     SELECT * FROM contexts
# """)
# contexts = cur.fetchall()

# print("contexts: \n context_uri, context_name")
# for i in contexts:
#     print(i)

# # print all the listenings from the database
# cur.execute("""
#     SELECT * FROM listenings
# """)
# listenings = cur.fetchall()

# print("listenings: \n user_id, track_id, context_id, timestamp, track_listens_count")
# for i in listenings:
#     print(i)


# # Commit the changes to the database.
# conn.commit()

# # Close the database connection.
# conn.close()


In [8]:
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
#     # Connect to the SQLite database.
#     cur = conn.cursor()
#     # create the users table with user_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_id INTEGER PRIMARY KEY,
#             user_uri TEXT NOT NULL,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     # create the tracks table with track_id as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_id INTEGER PRIMARY KEY,
#             track_uri TEXT NOT NULL,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     # create the contexts table with context_uri as the primary key
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # create the listenings table with user_id and track_id as the foreign keys
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id INTEGER NOT NULL,
#             track_id INTEGER NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL,
#             FOREIGN KEY (user_id) REFERENCES users (user_id),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # add user to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         # get the user_id of the inserted user
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             SELECT ?, ?, ? WHERE NOT EXISTS (
#                 SELECT 1 FROM users WHERE user_uri = ? AND user_name = ? AND user_imageUrl = ?
#             )
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl'], friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        
#         result = cur.fetchone()
#         if result:
#             user_id = result[0]

#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # get the track_id of the inserted track
#         cur.execute("""
#             SELECT track_id FROM tracks WHERE track_uri = ?
#         """, (friend['track']['uri'],))
#         track_id = cur.fetchone()[0]
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (user_id, track_id))
        
#         # previous_timestamps = cur.fetchall()
        
#     users = cur.fetchall()

#     for i in users:
#         print(i)

# # No need to explicitly commit or close the connection as it is handled by the context manager.


In [9]:
# from datetime import datetime
# import time
# import sqlite3
# import main
# import json

# # milliseconds to datetime
# def milliseconds_to_datetime(milliseconds):
#     return datetime.fromtimestamp(milliseconds / 1000.0)

# # get the friends activity json object from the main.py
# friends_activity_json = main.get_friends_activity_json()

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # Use a context manager to connect to the SQLite database.
# with sqlite3.connect('friends_activity.sqlite') as conn:
#     # Connect to the SQLite database.
#     cur = conn.cursor()

#     # Check if the table exists.
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS users (
#             user_uri TEXT PRIMARY KEY,
#             user_name TEXT NOT NULL,
#             user_imageUrl TEXT NOT NULL
#         )
#     ''')

#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS tracks (
#             track_uri TEXT PRIMARY KEY,
#             track_name TEXT NOT NULL,
#             track_imageUrl TEXT NOT NULL,
#             track_album_uri TEXT NOT NULL,
#             track_album_name TEXT NOT NULL,
#             track_artist_uri TEXT NOT NULL,
#             track_artist_name TEXT NOT NULL
#         )
#     ''')

#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS contexts (
#             context_uri TEXT PRIMARY KEY,
#             context_name TEXT NOT NULL
#         )
#     ''')

#     # Add a column for timestamps to the listenings table
#     cur.execute('''
#         CREATE TABLE IF NOT EXISTS listenings (
#             user_id TEXT NOT NULL,
#             track_id TEXT NOT NULL,
#             context_id TEXT NOT NULL,
#             timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
#             track_listens_count INTEGER NOT NULL,
#             FOREIGN KEY (user_id) REFERENCES users (user_uri),
#             FOREIGN KEY (track_id) REFERENCES tracks (track_uri),
#             FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
#         )
#     ''')

#     # add friends activity to the database
#     for friend in friends_activity_json['friends']:
#         # add user to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
#             VALUES (?, ?, ?)
#         """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
#         # add track to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
#             VALUES (?, ?, ?, ?, ?, ?, ?)
#         """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
#         # add context to the database
#         cur.execute("""
#             INSERT OR IGNORE INTO contexts (context_uri, context_name)
#             VALUES (?, ?)
#         """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        
#         # check if the user has listened to the same track before
#         cur.execute("""
#             SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
#         """, (friend['user']['uri'], friend['track']['uri']))
        
#         previous_timestamps = cur.fetchall()
        
#         # if yes, update the timestamp column with the new value and increment the track_listens_count column by 1
#         if previous_timestamps:
#             cur.execute("""
#                 UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ?
#             """, (friend['timestamp'], friend['user']['uri'], friend['track']['uri']))
        
#         # if no, insert a new row with the user_id, track_id, context_id, timestamp and track_listens_count columns
#         else:
#             cur.execute("""
#                 INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#                 VALUES (?, ?, ?, ?, ?)
#             """, (friend['user']['uri'], friend['track']['uri'], friend['track']['context']['uri'], friend['timestamp'], 1))

#     # get all the users from the database
#     cur.execute("""
#         SELECT * FROM users
#     """)
#     users = cur.fetchall()

#     for i in users:
#         print(i)

# # No need to explicitly commit or close the connection as it is handled by the context manager.


In [25]:
print(len("The blueprint is on a large, brownish yellow sheet of paper. The paper is slightly wrinkled and has a few folds in it. The drawing of the spaceship is in blue ink. The spaceship is a large, sleek, metallic craft. It is shaped like a teardrop, with a long, pointed nose and a wide, flat tail. The ship is covered in strange symbols and markings, which are written in an alien language that is unknown to humans. The text on the blueprint is also in blue ink."))

457


In [10]:
# friends_activity_json

In [11]:

# # Create a SQLite database file.
# conn = sqlite3.connect('friends_activity.sqlite')

# # Connect to the SQLite database.
# cur = conn.cursor()


# # select all data from listenings table
# # cur.execute("SELECT * FROM listenings")

# # adding my_dict to listenings table
# cur.execute("""
#     INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
#     VALUES (?, ?, ?, ?, ?)
# """, (my_dict['user']['uri'], my_dict['track']['uri'], my_dict['context']['uri'], my_dict['timestamp'], 1))

# # commit the changes
# conn.commit()

# # close the connection
# conn.close()


# # If the result is None, create the table.

# # # create users table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS users (
# #         user_id INTEGER PRIMARY KEY,
# #         user_uri TEXT,
# #         user_name TEXT,
# #         user_imageUrl TEXT
# #     )
# # """)

# # # create tracks table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS tracks (
# #         track_id INTEGER PRIMARY KEY,
# #         track_uri TEXT,
# #         track_name TEXT,
# #         track_imageUrl TEXT,
# #         track_album_uri TEXT,
# #         track_album_name TEXT,
# #         track_artist_uri TEXT,
# #         track_artist_name TEXT
# #     )
# # """)

# # # create contexts table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS contexts (
# #         context_id INTEGER PRIMARY KEY,
# #         context_uri TEXT,
# #         context_name TEXT
# #     )
# # """)

# # # create listenings table if not exists
# # cur.execute("""
# #     CREATE TABLE IF NOT EXISTS listenings (
# #         listening_id INTEGER PRIMARY KEY,
# #         user_id INTEGER,
# #         track_id INTEGER,
# #         context_id INTEGER,
# #         timestamp INTEGER,
# #         track_listens_count INTEGER,
# #         FOREIGN KEY (user_id) REFERENCES users (user_id),
# #         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
# #         FOREIGN KEY (context_id) REFERENCES contexts (context_id)
# #     )
# # """)


In [12]:
# my_dict = {'timestamp': 1682175987406,
#    'user': {'uri': 'spotify:user:sonemic.com',
#     'name': 'sonemic.com',
#     'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
#    'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
#     'name': 'Revolve',
#     'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46',
#     'album': {'uri': 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw',
#      'name': 'Stoner Witch'},
#     'artist': {'uri': 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3',
#      'name': 'Melvins'},
#     'context': {'uri': 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S',
#      'name': 'Front Page Features | Sonemic Selects',
#      'index': 0}}}

In [13]:

# # create a cursor object
# cur = conn.cursor()

# # select all data from users table
# cur.execute("SELECT * FROM listenings")

# # fetch all rows as a list of tuples
# rows = cur.fetchall()
# print(rows)

# # loop through each row and print it
# for row in rows:
#     print(row)

# # # close the cursor and connection objects
# # cur.close()
# # conn.close()

In [30]:

# # Check if the table exists.
# cur.execute('''
#         SELECT name
#         FROM sqlite_master
#         WHERE type = 'table'
#         AND name = 'friends_activity'
#         ''')

# # Store the result of fetchone() in a variable.
# result = cur.fetchone()

# # If the result is None, create the table.
# if result is None:
#   print('Creating table...')
#   cur.execute('''
#         CREATE TABLE friends_activity (
#           timestamp INTEGER,
#           user_uri TEXT,
#           user_name TEXT,
#           user_imageUrl TEXT,
#           track_uri TEXT,
#           track_name TEXT,
#           track_imageUrl TEXT,
#           track_album_uri TEXT,
#           track_album_name TEXT,
#           track_artist_uri TEXT,
#           track_artist_name TEXT,
#           track_context_uri TEXT,
#           track_context_name TEXT,
#           track_context_index INTEGER,
#           track_listens_count INTEGER
#         )
#         ''')

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # fetch result from the database
# # result = cur.execute('''SELECT * FROM friends_activity''')
# result = cur.fetchall()


# # print the result
# for row in result:
#     print(row)


# # Insert the data into the table.
# for friend in friends_activity_json['friends']:
#   cur.execute(''' INSERT INTO friends_activity (timestamp,
#                                 user_uri,
#                                 user_name,
#                                 track_uri,
#                                 track_name,
#                                 track_imageUrl,
#                                 track_album_uri,
#                                 track_album_name,
#                                 track_artist_uri,
#                                 track_artist_name,
#                                 track_context_uri,
#                                 track_context_name,
#                                 track_listens)
#   VALUES (?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?, ?)''',
#        (friend['timestamp'],
#         friend['user']['uri'],
#         friend['user']['name'],
#         friend['user']['imageUrl'],
#         friend['track']['uri'],
#         friend['track']['name'],
#         friend['track']['imageUrl'],
#         friend['track']['album']['uri'],
#         friend['track']['album']['name'],
#         friend['track']['artist']['uri'],
#         friend['track']['artist']['name'],
#         friend['track']['context']['uri'],
#         friend['track']['context']['name'],
#         # friend['track']['context']['index'],
#         ))


# Close the connection to the SQLite database.
# conn.close()

In [14]:
# cur.execute('''SELECT * FROM friends_activity''')
# rows = cur.fetchall()
# # print(rows)
# # print(rows)
# for i in rows:
#     print(i)

In [19]:
rows = cur.fetchall()
print(rows)

[]


In [20]:
print(rows)
for i in rows:
    print(i)

[]


In [15]:
# friends_activity_json

In [16]:
# data = (friend['timestamp'],
#         friend['user']['uri'],
#         friend['user']['name'],
#         friend['user']['imageUrl'],
#         friend['track']['uri'],
#         friend['track']['name'],
#         friend['track']['imageUrl'],
#         friend['track']['album']['uri'],
#         friend['track']['album']['name'],
#         friend['track']['artist']['uri'],
#         friend['track']['artist']['name'],
#         # friend['track']['context']['uri'],
#         # friend['track']['context']['name'],
#         # friend['track']['context']['index']
#         )
# for i in data:
#   print(i)

In [17]:
# friends_activity_json